# Lab. 12: Robust Optimization

## Introduction

#### <u>In this lab, we will see some applications of robust optimization, namely a modified version of the Knapsack 0/1 problem, and the portfolio optimization problem.</u>

Your job in this lab is to implement the missing functions, and study how different functions lead to different outcomes from both the point of view of the objective value and the probability of violating the constraints of the problem.

The examples are taken from https://xiongpengnus.github.io/rsome/ro_rsome, using the RSOME library for robust optimization.

In [ ]:
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import rsome as rso
from rsome import grb_solver as grb
from rsome import ro

## Exercises

### Exercise 1/2: Modified Knapsack 0/1 Problem

In this exercise, we will solve the Knapsack problem (seen in the previous labs), slightly modified in order to have uncertainties about the volumes of the items.

The uncertainty about the volumes is not the same for all the items. They are defined by  δ , defined as a fraction of the size of the volumes of the items.

In this exercise, you are asked to implement the definition of the uncertainty set in order to have both an ellipsoidal uncertainty set and a finite uncertainty set.

#### Task
Implement different sizes for the ellipsoid and different interval for the finite set and compare the objective values and the probability of violating the constraints with the different setups.

In [ ]:
from typing import Any, Callable

from numpy.typing import NDArray

items = [
    {"name": "apple", "value": 1, "volume": 2},
    {"name": "pear", "value": 2, "volume": 2},
    {"name": "banana", "value": 2, "volume": 2},
    {"name": "watermelon", "value": 5, "volume": 10},
    {"name": "orange", "value": 3, "volume": 2},
    {"name": "avocado", "value": 3, "volume": 2},
    {"name": "blueberry", "value": 3, "volume": 1},
    {"name": "coconut", "value": 4, "volume": 3},
    {"name": "cherry", "value": 2, "volume": 1},
    {"name": "apricot", "value": 1, "volume": 1},
]
N = len(items)
C = 10

c = np.array([i["value"] for i in items]).flatten()  # profit coefficients
w = np.array([i["volume"] for i in items]).flatten()  # weight coefficients

delta = 0.2 * w  # maximum deviations


def robust(
    get_uncertainty_set: Callable[[Any, Any], Any], r: int | list[int] | list[float]
) -> tuple[int, NDArray[np.float64]]:
    """
    The function robust implements the robust optimization model, given the budget of uncertainty r
    """

    model = ro.Model("robust")
    x = model.dvar(N, vtype="B")  # Boolean variable x (0: leave, 1: keep)
    z = model.rvar(N)  # Random variable

    # Uncertainty set
    z_set = get_uncertainty_set(z, r)
    # z_set = (abs(z) <= 1, rso.norm(z, 1) <= r)

    # Maximize the value of the knapsack (i.e., the dot product between the values and x)
    model.max(c @ x)

    # Add constraint: the maximum (uncertain) weight is smaller than the budget
    # in ellipsoidal uncertainty set, 'M' is 'delta' and 'u' is 'z'
    # in finite uncertainty set, p1 = w + z1*delta => z1 = (p1 - w)/delta
    model.st(((w + z * delta) @ x <= C).forall(z_set))
    # consider the optimization with constraint "Ax <= b", here A = w + z*delta

    # Solve
    model.solve(grb, display=False)

    return model.get(), x.get()


def sim(x_sol: NDArray[np.float64], zs: NDArray[np.float64]):
    """
    The function sim is for calculating the probability of violation via simulations.
        x_sol: solution of the Knapsack problem
        zs: random sample of the random variable z
    """

    ws = w + zs * delta  # random samples of uncertain weights

    return (ws @ x_sol > C).mean()


def ellipsoidal_uncertainty_set(z: Any, r: int) -> tuple[Any]:
    """
    Define an uncertainty set. See the following sources:
    - Ellipsoidal: https://xiongpengnus.github.io/rsome/ro_rsome#section2.2
    - Finite Uncertainty Set
    """
    z_set = rso.norm(z, 2) <= r  # type: ignore

    return z_set


def finite_uncertainty_set(z: Any, r: list[int]) -> tuple[Any]:
    # p1 = w + z1*delta => z1 = (p1 - w)/delta
    z_set = (z <= r[0], z >= r[0]) or (z <= r[1], z >= r[1]) or (z <= r[2], z >= r[2])
    return z_set  # type: ignore


def finite_uncertainty_set_big(z: Any, r: list[int]) -> tuple[Any]:
    z_set = (
        (z <= r[0], z >= r[0])
        or (z <= r[1], z >= r[1])
        or (z <= r[2], z >= r[2])
        or (z <= r[3], z >= r[3])
        or (z <= r[4], z >= r[4])
        or (z <= r[5], z >= r[5])
    )
    print(z_set)
    return z_set  # type: ignore


def finite_uncertainty_set_hardcoded(z: Any, r: int) -> tuple[Any]:
    z_set = (
        (z <= 0, z >= 0)
        or (z <= 1, z >= 1)
        or (z <= 2, z >= 2)
        or (z <= 3, z >= 3)
        or (z <= 4, z >= 4)
        or (z <= 5, z >= 5)
    )
    return z_set  # type: ignore

In [ ]:
@dataclass(frozen=True)
class Result:
    r: int
    solution: NDArray[np.float64]
    prob_violation: float
    objective_value: float

In [ ]:
def plot_knapsack_results(results: list[Result]) -> None:
    _, ax = plt.subplots(
        1,
        1,
    )
    for res in results:
        ax.scatter(res.r, res.prob_violation, color="blue")
        ax.annotate(
            f"{res.objective_value:.0f}",
            (res.r, res.prob_violation),
            textcoords="offset points",
            xytext=(0, 10),
            ha="center",
        )
    ax.set_xlabel("r")
    ax.set_ylabel("Probability of violation")
    ax.set_ylim(0, 1)
    plt.show()

In [ ]:
num_samples = 20000
rs = np.arange(0, 5, 0.5)
results: list[Result] = []
for r in rs:
    zs = np.random.uniform(-1, 1, (num_samples, N))  # Generate random samples for z
    objective_value, solution = robust(ellipsoidal_uncertainty_set, r)
    prob_violation = sim(solution, zs)
    results.append(Result(r, solution, prob_violation, objective_value))
plot_knapsack_results(results)

In [ ]:
# p1 = w + r*delta => z1 = (p1 - w)/delta
rs = [
    [-5, -4, -3],
    [-4, -3, -2],
    [-3, -2, -1],
    [-2, -1, 0],
    [-1, 0, 1],
    [0, 1, 2],
    [1, 2, 3],
    [2, 3, 4],
    [3, 4, 5],
]
for r in rs:
    zs = np.random.uniform(-1, 1, (num_samples, N))  # Generate random samples for z
    objective_value, solution = robust(finite_uncertainty_set, r)
    prob_violation = sim(solution, zs)
    print("r: ", r)
    print("Objective value: ", objective_value)
    print("Probability of violation: ", prob_violation)
    print("----------------------")

In [ ]:
# p1 = w + r*delta => z1 = (p1 - w)/delta
rs = [
    [-1.0, -0.8, -0.6],
    [-0.8, -0.6, -0.4],
    [-0.6, -0.4, -0.2],
    [-0.4, -0.2, 0.0],
    [-0.2, 0.0, 0.2],
    [0.0, 0.2, 0.4],
    [0.2, 0.4, 0.6],
    [0.4, 0.6, 0.8],
    [0.6, 0.8, 1.0],
]
for r in rs:
    zs = np.random.uniform(-1, 1, (num_samples, N))  # Generate random samples for z
    objective_value, solution = robust(finite_uncertainty_set, r)
    prob_violation = sim(solution, zs)
    print("r: ", r)
    print("Objective value: ", objective_value)
    print("Probability of violation: ", prob_violation)
    print("----------------------")

In [ ]:
num_samples = 20000
zs = np.random.uniform(-1, 1, (num_samples, N))  # Generate random samples for z

objective_value, solution = robust(finite_uncertainty_set, [0, 0, 0])
prob_violation = sim(solution, zs)

print("Content of the knapsack:")
for i, value in enumerate(solution):
    if value:
        print(f'\t{items[i]["name"]}')
print(f"Total value: {objective_value}. Probability of violation: {prob_violation}")

### Exercise 2/2: Robust Portfolio Optimization

In this problem, we want to build a portfolio (e.g., of stocks), by using robust approaches.

To be more specific, in this problem we have a set of fictionary stocks, each of which has different means and deviations for the returns.

#### TASK
Your job here is to implement a box uncertainty set to robustly optimize the portfolio. Try different values for the box in order to study how the uncertainty affects the objective value of and the number of different stocks chosen.

In [ ]:
n = 10
stocks = {
    f"Company {chr(65+i)}": {
        "Mean": np.around(np.random.uniform(0.9, 1.1), 2),
        "Deviation": np.around(np.random.uniform(0.1, 0.3), 2),
    }
    for i in range(n)
}

stocks = {
    "Company A": {"Deviation": 0.17, "Mean": 0.97},
    "Company B": {"Deviation": 0.22, "Mean": 0.92},
    "Company C": {"Deviation": 0.28, "Mean": 0.95},
    "Company D": {"Deviation": 0.22, "Mean": 1.04},
    "Company E": {"Deviation": 0.23, "Mean": 0.97},
    "Company F": {"Deviation": 0.24, "Mean": 1.08},
    "Company G": {"Deviation": 0.27, "Mean": 1.05},
    "Company H": {"Deviation": 0.12, "Mean": 1.01},
    "Company I": {"Deviation": 0.24, "Mean": 1.03},
    "Company J": {"Deviation": 0.29, "Mean": 0.95},
}


def portfolio_optimization(
    get_uncertainty_set: Callable[[Any, Any, Any], Any],
    lower: int | float,
    upper: int | float,
) -> tuple[int, NDArray[np.float64]]:
    p = np.array([stocks[s]["Mean"] for s in stocks])  # mean returns
    delta = np.array([stocks[s]["Deviation"] for s in stocks])  # deviations of returns

    model = ro.Model()
    x = model.dvar(n)  # fractions of investment
    z = model.rvar(n)  # random variables

    z_set = get_uncertainty_set(z, lower, upper)

    model.maxmin((p + delta * z) @ x, z_set)  # the max-min objective

    model.st(sum(x) == 1)  # type: ignore    # summation of x is one
    model.st(x >= 0)  # x is non-negative

    model.solve(grb)  # solve the model by Gurobi
    return model.get(), x.get()


def get_uncertainty_set_po(
    z: Any, lower: int | float, upper: int | float
) -> tuple[Any]:
    """
    Return a box uncertainty set
    (see https://xiongpengnus.github.io/rsome/example_ro_inv).

    Try different values for the size of the box.
    """
    z_set = (z <= upper, z >= lower)
    return z_set  # type: ignore


def get_uncertainty_set_po_norm(
    z: Any, lower: int | float, upper: int | float
) -> tuple[Any]:
    """
    Return a box uncertainty set
    (see https://xiongpengnus.github.io/rsome/example_ro_inv).

    Try different values for the size of the box.
    """
    z_set = (
        rso.norm(z, np.infty) <= 1,  # type: ignore
        rso.norm(z, 1) <= upper,  # type: ignore
    )  # type: ignore
    return z_set  # type: ignore

In [ ]:
def print_portfolio_results(x_sol: NDArray[np.float64]) -> None:
    print("Content of the portfolio:")
    for i, value in enumerate(x_sol):
        if value != 0:
            company = f"Company {chr(65+i)}"
            print(
                f'\t{company} (Mean {stocks[company]["Mean"]}, Deviation {stocks[company]["Deviation"]}): {value*100:.2f}%'
            )

In [ ]:
def plot_portfolio_results(x_sol: NDArray[np.float64]) -> None:
    _, ax = plt.subplots(
        1,
        1,
    )
    ax.bar(
        [s.split(" ")[1] for s in stocks], x_sol  # Stock names
    )  # Investment fractions
    ax.set_xlabel("Stocks")
    ax.set_ylabel("Investment fractions")
    plt.show()

In [ ]:
obj_val, x_sol = portfolio_optimization(get_uncertainty_set_po, -0.25, 0.25)
print("Objective value: {0:0.4f}".format(obj_val))
print_portfolio_results(x_sol)
plot_portfolio_results(x_sol)

In [ ]:
# positive bounds -> I take the stock with the highest return
obj_val, x_sol = portfolio_optimization(get_uncertainty_set_po, 0.8, 1.2)
print("Objective value: {0:0.4f}".format(obj_val))
print_portfolio_results(x_sol)
plot_portfolio_results(x_sol)

In [ ]:
# negative bounds -> I take the stock with the lowest deviation (p + delta*z)
obj_val, x_sol = portfolio_optimization(get_uncertainty_set_po, -1.2, -0.8)
print("Objective value: {0:0.4f}".format(obj_val))
print_portfolio_results(x_sol)
plot_portfolio_results(x_sol)

In [ ]:
norms = np.arange(0, 2.1, 0.5)
for norm in norms:
    obj_val, x_sol = portfolio_optimization(get_uncertainty_set_po_norm, 0, norm)
    print("Norm: ", norm)
    print("Objective value: {0:0.4f}".format(obj_val))
    print_portfolio_results(x_sol)
    plot_portfolio_results(x_sol)